# understand simulated data

In [1]:
import pandas as pd

sim_s = pd.read_pickle("../simulated_data/seed_0_sim.pkl")
org_s = pd.read_pickle("../processed_data/student_trajectories.pkl")

sim_n = pd.read_pickle("../simulated_data/seed_0_sim_narr.pkl")
org_n = pd.read_pickle("../processed_data/narrative_trajectories.pkl")

In [34]:
# static features will remain the same. so will the initial states.
# the only thing that we need to look through is the actions taken in each tune

d = org_s.groupby('episode')['action'].apply(list).reset_index()
d['org_len'] = d.apply(lambda x: len(x['action']), axis=1)
d['action'] = d.apply(lambda x: x['action'][:500] if x['org_len']>=500 \
    else x['action'] + [0]*(500 - len(x['action'])), axis=1)
d['len'] = d.apply(lambda x: len(x['action']), axis=1)
d

,episode,action,org_len,len
0,100-0001,"[0, 0, 1, 0, 0, 14, 9, 9, 9, 9, 9, 9, 9, 9, 0,...",560,500
1,100-0003,"[1, 1, 16, 9, 0, 9, 12, 9, 2, 2, 2, 0, 13, 6, ...",410,500
2,100-0004,"[1, 1, 1, 1, 1, 15, 9, 1, 1, 14, 9, 6, 6, 6, 1...",417,500
3,100-0005,"[1, 14, 9, 9, 9, 9, 9, 9, 14, 6, 9, 6, 6, 6, 6...",508,500
4,100-0006,"[0, 7, 7, 0, 1, 14, 9, 9, 9, 9, 9, 9, 9, 9, 6,...",386,500
...,...,...,...,...
394,100-0513,"[1, 6, 6, 6, 6, 0, 14, 0, 9, 12, 9, 6, 14, 9, ...",324,500
395,100-0515,"[1, 0, 14, 9, 9, 9, 9, 9, 6, 1, 3, 10, 3, 10, ...",401,500
396,100-0516,"[1, 15, 9, 6, 6, 6, 6, 6, 1, 0, 1, 1, 13, 13, ...",410,500
397,100-0827,"[1, 0, 14, 9, 1, 6, 1, 1, 14, 9, 9, 9, 9, 9, 9...",400,500


In [35]:
d2 = sim_s.groupby('episode')['action'].apply(list).reset_index()
d2['org_len'] = d2.apply(lambda x: len(x['action']), axis=1)
d2['action'] = d2.apply(lambda x: x['action'][:500] if x['org_len']>=500 \
    else x['action'] + [0]*(500 - len(x['action'])), axis=1)
d2['len'] = d2.apply(lambda x: len(x['action']), axis=1)
d2


,episode,action,org_len,len
0,0,"[8, 0, 0, 8, 0, 9, 9, 9, 4, 0, 14, 8, 8, 9, 9,...",500,500
1,1,"[0, 0, 0, 9, 9, 14, 9, 0, 8, 8, 6, 9, 8, 14, 9...",500,500
2,100,"[1, 8, 8, 8, 9, 8, 10, 6, 10, 8, 6, 11, 12, 9,...",500,500
3,1000,"[9, 0, 14, 0, 0, 8, 0, 0, 0, 8, 8, 0, 12, 9, 8...",500,500
4,1001,"[13, 10, 8, 6, 1, 8, 0, 8, 10, 8, 14, 9, 9, 9,...",500,500
...,...,...,...,...
4689,995,"[0, 8, 9, 8, 8, 10, 8, 0, 9, 8, 0, 0, 9, 9, 8,...",500,500
4690,996,"[0, 0, 0, 9, 0, 0, 2, 8, 0, 8, 8, 9, 8, 9, 9, ...",500,500
4691,997,"[0, 0, 8, 8, 0, 8, 9, 8, 10, 8, 10, 9, 1, 0, 8...",500,500
4692,998,"[6, 0, 7, 8, 0, 8, 8, 10, 8, 9, 8, 8, 13, 0, 8...",500,500


In [94]:
from collections import Counter

combine = 99
combine_count = 0
len_org = len(d)
len_sim = len(d2)

actions_org = []
actions_sim = []
for position in range(500):
    if combine_count < combine:
        org = list(d.apply(lambda x: x['action'][position], axis=1).values)
        sim = list(d2.apply(lambda x: x['action'][position], axis=1).values)
        actions_org += org
        actions_sim += sim
        combine_count += 1
        continue
    else:
        combine_count = 0

    act_org = Counter(actions_org)
    for act, item in act_org.items():
        act_org[act] = round(item/(len_org*combine)*100.00, 2)
    for act in range(18):
        act_org[act] = act_org.get(act, 0.0)
    act_org = sorted(act_org.items())

    act_sim = Counter(actions_sim)
    for act, item in act_sim.items():
        act_sim[act] = round(item/(len_sim*combine)*100.0, 2)
    for act in range(18):
        act_sim[act] = act_sim.get(act, 0.0)
    act_sim = sorted(act_sim.items())

    for action, (org, sim) in enumerate(zip(act_org, act_sim)):
        if abs(org[1] - sim[1]) > 10.0:
            print("Position {0} | Action {1} | Org {2} | Sim {3}".format(position+1, action, org[1], sim[1]))
        if org[1]>10:
            print("Heavy action {0} with percentage {1} | sim {2}".format( action, org[1], sim[1]))
    actions_org = []
    actions_sim = []

Position 100 | Action 1 | Org 15.49 | Sim 1.4
Heavy action 1 with percentage 15.49 | sim 1.4
Position 100 | Action 6 | Org 15.35 | Sim 4.05
Heavy action 6 with percentage 15.35 | sim 4.05
Position 100 | Action 8 | Org 4.65 | Sim 26.37
Heavy action 9 with percentage 31.33 | sim 23.02
Position 200 | Action 1 | Org 13.93 | Sim 1.39
Heavy action 1 with percentage 13.93 | sim 1.39
Position 200 | Action 3 | Org 11.34 | Sim 0.54
Heavy action 3 with percentage 11.34 | sim 0.54
Heavy action 6 with percentage 11.36 | sim 3.11
Position 200 | Action 8 | Org 6.05 | Sim 23.05
Heavy action 9 with percentage 20.77 | sim 18.08
Heavy action 10 with percentage 11.26 | sim 5.31
Position 200 | Action 14 | Org 2.83 | Sim 16.53
Position 300 | Action 1 | Org 12.84 | Sim 1.32
Heavy action 1 with percentage 12.84 | sim 1.32
Position 300 | Action 3 | Org 10.84 | Sim 0.61
Heavy action 3 with percentage 10.84 | sim 0.61
Heavy action 6 with percentage 10.78 | sim 2.95
Position 300 | Action 8 | Org 9.4 | Sim 24.17
H